# Collecting Data From the Riot Games API

This notebook walks you through the Riot Games API and how to collect the needed data from the the different endpoints. To access the Riot Games API you have to log in through your Riot Games account and regenerate your assigned API key (expires every 24h). The link to the Riot Games API is below.

https://developer.riotgames.com/

In [2]:
# Import Needed Packages
import requests as re
import pandas as pd

In [4]:
# Example Header:
# Headers have to be retrieved from the Riot API and is unique every time a development key is regenerated 

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77",
    "Accept-Language": "en-CA,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-ebf4231e-271a-42b0-8b8e-7bd1cb5510d0"
}

## Getting A Summoner's PUUID

The easiest way to start collecting PUUIDs is to find the in-game name of players. You can choose your own name, the names of summoners in your games, random summoners on the LoL leaderboard, or your favourite streamers or pro players. The links below are to different online leaderboards that you can change the regions of.

https://na.op.gg/leaderboards/tier?hl=en_US

https://u.gg/leaderboards/ranking?region=na1


This it the link to the Summoner API Endpoints:

https://developer.riotgames.com/apis#summoner-v4/GET_getByPUUID

In [8]:
'''
This is an example uses the EU Pro-Player Odoamne. You can change the 'euw1' in the url to the LoL server of the summoner
that you are searching for. This endpoint when converted to JSON returns a dictionary.
'''

url_player_example = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Odoamne'

# Send a request to the API and return the result in JSON format.
player_example = re.get(url_player_example, headers=headers).json()

# Print the dictionary
print(player_example)
print()
# Access and print the puuid
print('puuid: ', player_example['puuid'])

{'id': '-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww', 'accountId': 'PxLMdMt6DGPnQCmvIbZRChUenr-jMSXlYC5Ki0fYTXgHTQ', 'puuid': 'lwqnB8N0S-0fIzHi-MPs5w8mglAim9K37Y0jXGCvB9Qt7TUj2aYbxabeIzn73okCKncBwUUkhXNcuA', 'name': 'Odoamne', 'profileIconId': 764, 'revisionDate': 1659192400000, 'summonerLevel': 419}

puuid:  lwqnB8N0S-0fIzHi-MPs5w8mglAim9K37Y0jXGCvB9Qt7TUj2aYbxabeIzn73okCKncBwUUkhXNcuA


## Get the MatchIDs of the Last 20 Games a Summoner Has Played

Using the Matches API (https://developer.riotgames.com/apis#match-v5), we need to get the match ids using a summoner's puuid.

In [10]:
'''
This is an example uses the EU Pro-Player Odoamne. You can change the 'europe' in the url to the LoL region of the summoner
that you are searching for (europe, asia, americas).
'''

# Puuid is retrieved from the above example
puuid =  player_example['puuid']

# Create empty list for all matchIDs
matchID_list = []

# Send a request to get the matchIDs for the latest 20 games played and convert result to JSON
url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?start=0&count=20'
matchIDs = re.get(url, headers=headers).json()

# Access each matchID
for matchID in matchIDs:
    # Append the matchID to the master list EUW_matchID
    matchID_list.append(matchID)

print(matchID_list)

['EUW1_6003315724', 'EUW1_6003360538', 'EUW1_6003116563', 'EUW1_6002003725', 'EUW1_6001879312', 'EUW1_6001730553', 'EUW1_6001647198', 'EUW1_6001565779', 'EUW1_6001582598', 'EUW1_6001039285', 'EUW1_6000965422', 'EUW1_6000961149', 'EUW1_6000924070', 'EUW1_6000819929', 'EUW1_5999960256', 'EUW1_5999534027', 'EUW1_5999540762', 'EUW1_5999472937', 'EUW1_5999336570', 'EUW1_5999291641']


## Collect the Match Information

This step also uses the matches endpoint (https://developer.riotgames.com/apis#match-v5), but we will get the ,atch information by match id. The desired columns for this project are:

- Match ID
- Blue(0) and Red(1) team picks (Champion ID)
- Blue(0) and Red(1) team bans (BAN Champion ID)
- First objectives (Kill, Dragon, Rift Herald, Tower, Baron, Inhibitor)
- Blue(0) and Red(1) team total objective kills
- Winner

It is important to note that game convention states that:
- Summoner 1 plays TOP
- Summoner 2 plays JGL
- Summoner 3 plays MID
- Summoner 4 plays BOT
- Summoner 5 plays SUP

In [21]:
# Create an empty dataframe with the desired columns
matches = pd.DataFrame(columns=['MATCH_ID', 
       'BSUM1_ID', 'BTOP_CHAMP_ID', 'BSUM2_ID','BJGL_CHAMP_ID', 'BSUM3_ID', 'BMID_CHAMP_ID', 
       'BSUM4_ID', 'BBOT_CHAMP_ID', 'BSUM5_ID', 'BSUP_CHAMP_ID', 
       'BBAN1', 'BBAN2', 'BBAN3', 'BBAN4','BBAN5', 
       'RSUM1_ID', 'RTOP_CHAMP_ID', 'RSUM2_ID', 'RJGL_CHAMP_ID', 'RSUM3_ID', 'RMID_CHAMP_ID', 
       'RSUM4_ID', 'RBOT_CHAMP_ID', 'RSUM5_ID', 'RSUP_CHAMP_ID', 
       'RBAN1', 'RBAN2', 'RBAN3', 'RBAN4', 'RBAN5', 'FIRST_BLOOD','BCHAMP_KILLS' , 'RCHAMP_KILLS',
       'FIRST_DRAG', 'BDRAG_KILLS' , 'RDRAG_KILLS', 
       'FIRST_HERALD', 'BHERALD_KILLS', 'RHERALD_KILLS', 
       'FIRST_TOWER' ,'BTOWER_KILLS' , 'RTOWER_KILLS',
       'FIRST_BARON','BBARON_KILLS' , 'RBARON_KILLS',
       'FIRST_INHIB','BINHIB_KILLS' , 'RINHIB_KILLS', 
       'WINNER'])

In [22]:
# Send a request to the Riot Games API to get the match info for each matchID
for matchID in matchID_list:
    url = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + matchID
    match_info = re.get(url, headers=headers).json()

    # Create a temporary DataFrame
    match = pd.DataFrame()

    # Get the MatchID
    match['MATCH_ID'] = [match_info['metadata']['matchId']]

    # Get the Summoner ID and ChampionID for each member on the Blue Team
    match['BSUM1_ID'] = [match_info['info']['participants'][0]['summonerId']]
    match['BTOP_CHAMP_ID'] = [match_info['info']['participants'][0]['championId']]

    match['BSUM2_ID'] = match_info['info']['participants'][1]['summonerId']
    match['BJGL_CHAMP_ID'] = match_info['info']['participants'][1]['championId']

    match['BSUM3_ID'] = match_info['info']['participants'][2]['summonerId']
    match['BMID_CHAMP_ID'] = match_info['info']['participants'][2]['championId']

    match['BSUM4_ID'] = match_info['info']['participants'][3]['summonerId']
    match['BBOT_CHAMP_ID'] = match_info['info']['participants'][3]['championId']

    match['BSUM5_ID'] = match_info['info']['participants'][4]['summonerId']
    match['BSUP_CHAMP_ID'] = match_info['info']['participants'][4]['championId']

    # Get the Champion ID for the 5 blue team bans
    match['BBAN1'] = match_info['info']['teams'][0]['bans'][0]['championId']
    match['BBAN2'] = match_info['info']['teams'][0]['bans'][1]['championId']
    match['BBAN3'] = match_info['info']['teams'][0]['bans'][2]['championId']
    match['BBAN4'] = match_info['info']['teams'][0]['bans'][3]['championId']
    match['BBAN5'] = match_info['info']['teams'][0]['bans'][4]['championId']

    # Get the Summoner ID and ChampionID for each member on the Red Team
    match['RSUM1_ID'] = match_info['info']['participants'][5]['summonerId']
    match['RTOP_CHAMP_ID'] = match_info['info']['participants'][5]['championId']

    match['RSUM2_ID'] = match_info['info']['participants'][6]['summonerId']
    match['RJGL_CHAMP_ID'] = match_info['info']['participants'][6]['championId']

    match['RSUM3_ID'] = match_info['info']['participants'][7]['summonerId']
    match['RMID_CHAMP_ID'] = match_info['info']['participants'][7]['championId']

    match['RSUM4_ID'] = match_info['info']['participants'][8]['summonerId']
    match['RBOT_CHAMP_ID'] = match_info['info']['participants'][8]['championId']

    match['RSUM5_ID'] = match_info['info']['participants'][9]['summonerId']
    match['RSUP_CHAMP_ID'] = match_info['info']['participants'][9]['championId']

    # Get the Champion ID for the 5 red team bans
    match['RBAN1'] = match_info['info']['teams'][1]['bans'][0]['championId']
    match['RBAN2'] = match_info['info']['teams'][1]['bans'][1]['championId']
    match['RBAN3'] = match_info['info']['teams'][1]['bans'][2]['championId']
    match['RBAN4'] = match_info['info']['teams'][1]['bans'][3]['championId']
    match['RBAN5'] = match_info['info']['teams'][1]['bans'][4]['championId']

    # Assign who got First Blood and get the total kills for each team
    if match_info['info']['teams'][0]['objectives']['champion']['first'] == True:
        match['FIRST_BLOOD'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['champion']['first'] == True:
        match['FIRST_BLOOD'] = ['RED']
    else:
        match['FIRST_BLOOD'] = ['NONE']
    
    match['BCHAMP_KILLS'] = match_info['info']['teams'][0]['objectives']['champion']['kills']
    match['RCHAMP_KILLS'] = match_info['info']['teams'][1]['objectives']['champion']['kills']

    # Assign who got First Dragon and get the total dragon kills for each team
    if match_info['info']['teams'][0]['objectives']['dragon']['first'] == True:
        match['FIRST_DRAG'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['dragon']['first'] == True:
        match['FIRST_DRAG'] = ['RED']
    else:
        match['FIRST_DRAG'] = ['NONE']

    match['BDRAG_KILLS'] = match_info['info']['teams'][0]['objectives']['dragon']['kills']
    match['RDRAG_KILLS'] = match_info['info']['teams'][1]['objectives']['dragon']['kills']

    # Assign who got First Rift Herald and get the total rift herald kills for each team
    if match_info['info']['teams'][0]['objectives']['riftHerald']['first'] == True:
        match['FIRST_HERALD'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['riftHerald']['first'] == True:
        match['FIRST_HERALD'] = ['RED']
    else:
        match['FIRST_HERALD'] = ['NONE']
    
    match['BHERALD_KILLS'] = match_info['info']['teams'][0]['objectives']['riftHerald']['kills']
    match['RHERALD_KILLS'] = match_info['info']['teams'][1]['objectives']['riftHerald']['kills']

    # Assign who got First Tower and get the total tower kills for each team
    if match_info['info']['teams'][0]['objectives']['tower']['first'] == True:
        match['FIRST_TOWER'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['tower']['first'] == True:
        match['FIRST_TOWER'] = ['RED']
    else:
        match['FIRST_TOWER'] = ['NONE']

    match['BTOWER_KILLS'] = match_info['info']['teams'][0]['objectives']['tower']['kills']
    match['RTOWER_KILLS'] = match_info['info']['teams'][1]['objectives']['tower']['kills']

    # Assign who got First Baron and get the total baron kills for each team
    if match_info['info']['teams'][0]['objectives']['baron']['first'] == True:
        match['FIRST_BARON'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['baron']['first'] == True:
        match['FIRST_BARON'] = ['RED']
    else:
        match['FIRST_BARON'] = ['NONE']
    
    match['BBARON_KILLS'] = match_info['info']['teams'][0]['objectives']['baron']['kills']
    match['RBARON_KILLS'] = match_info['info']['teams'][1]['objectives']['baron']['kills']

    # Assign who got First Inhibitor and get the total inhibitor kills for each team
    if match_info['info']['teams'][0]['objectives']['inhibitor']['first'] == True:
        match['FIRST_INHIB'] = ['BLUE']
    elif match_info['info']['teams'][1]['objectives']['inhibitor']['first'] == True:
        match['FIRST_INHIB'] = ['RED']
    else:
        match['FIRST_INHIB'] = ['NONE']
    
    match['BINHIB_KILLS'] = match_info['info']['teams'][0]['objectives']['inhibitor']['kills']
    match['RINHIB_KILLS'] = match_info['info']['teams'][1]['objectives']['inhibitor']['kills']

    # Determine which team won and assign it to the winner ('target') variable
    if match_info['info']['teams'][0]['win'] == True:
        match['WINNER'] = ['BLUE']
    elif match_info['info']['teams'][1]['win'] == True:
        match['WINNER'] = ['RED']
    
    matches = pd.concat([matches, match])

In [23]:
matches

,MATCH_ID,BSUM1_ID,BTOP_CHAMP_ID,BSUM2_ID,BJGL_CHAMP_ID,BSUM3_ID,BMID_CHAMP_ID,BSUM4_ID,BBOT_CHAMP_ID,BSUM5_ID,...,FIRST_TOWER,BTOWER_KILLS,RTOWER_KILLS,FIRST_BARON,BBARON_KILLS,RBARON_KILLS,FIRST_INHIB,BINHIB_KILLS,RINHIB_KILLS,WINNER
0,EUW1_6003315724,C-KyVgTgtdpbzVfVKScAwMs8soZTGJJMt4o2PDIyX3nGINH1,79,SbkPONqoqVwY1oAz5Z2TNI0XV0XZM8NaSsRNO2p9uXNOaoA,104,C5OUqGm_Bz53_152eOHb0xnxWLkAIzgv-W5vBpdpXdYqSN3F,131,-Eke-I68hNPM1or754t1CAO0xvu0bblDkEaN2wIbP9bninY,29,hye6DxO7x7pS078UbKjKqZ-C_Ic-tkAnusQ07AI8ZOETlC...,...,RED,2,8,BLUE,1,1,RED,0,1,RED
0,EUW1_6003360538,-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww,58,yHR0BeOEB9oHyrKHrE8O05lzDk603T5JxfFmEPx36dc3CWA,91,Jg1j09bokAvG2pIH0SzTijxO3EvNBFsBAGHeF10SvHy3b_dE,163,up1BnQIZuihRtJ30koubgmvQ5ApnYKS5SEXuHRZ7fYCoy8r6,81,NO5LOqpuwWlNyfpR7ArfRHRGbxfn7MSNcyGe2zC27qZp4ez2,...,RED,2,7,NONE,0,0,NONE,0,0,RED
0,EUW1_6003116563,9jDWlsmQmP2alWxx92Rup0GgfSdMwDcRSB5aMQ89d_f7rNs,164,yHR0BeOEB9oHyrKHrE8O05lzDk603T5JxfFmEPx36dc3CWA,30,kEuynC1Rbb3YTlFPvyhQ78NpQ1EpvqdxFyOY7eaJY-_mHqMX,42,up1BnQIZuihRtJ30koubgmvQ5ApnYKS5SEXuHRZ7fYCoy8r6,29,ikkWHVa-xfimUksIOKccQF3WGN3r0K6hcy3-ziiMyg8uvPk,...,BLUE,2,3,RED,0,1,NONE,0,0,RED
0,EUW1_6002003725,-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww,113,LbagGIrqAB9pVUNO51qrQo_--GT5fNmWiRlXHLR8V7uXtxo,60,RYVNtsRyezqfrVQxWx966Ot5cHUuQI0GgXb8aLg_37tLCeo,31,X_pa7lUCrVFFwWp-qpcKtI0TcJf-FeQRxoe2Y72L20wxIf0,15,eQJdQpIaA7DX9dfeucVavwVdVI4aCrQVrYmc2sU55vS28mM,...,BLUE,10,1,BLUE,1,0,BLUE,2,0,BLUE
0,EUW1_6001879312,-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww,58,k2a5HoRnB083XF5CiSBpN2YAzi6wEC6nvuuynbTwdJ1-i_c,104,7nywNCOnXyibsU3cz-I62uZN3w2b5JtbfA4vE9_bWPShRBg,55,GoTdHb1N2H5H8aKnGttUR3UZbf_6y6KQk_SUx7Etj7pdHO8,119,F66NXGYFavcAokjrfJpUel2USWZkq8NHqpJTd_G27IMtrFI,...,RED,6,8,RED,0,1,RED,0,2,RED
0,EUW1_6001730553,t6vzJOdTrnFlbjiysfDD7eKSyUEC0z27laUu_qlZHythn8Y,24,KOdyW7xgmCkbSbNl9uGEOEu6RXrOzcBiIA2tmzGzdAshP30,78,l18PKfukA9g-ht9isV1mMVrEV6nhaAKlTercHfv84y6FtU...,7,DItXTeFPkWtK8I5fGxZxUQlV_HBmnzL_FQ-sZVorleornkU,221,qfbOKbPz0KKMmR4cvn0HDr0YOoiCdWFRS29wtV0e4WBA0FI,...,RED,6,8,RED,0,1,BLUE,1,1,RED
0,EUW1_6001647198,SIf8ZQVxODYq2nGPBcDa3llIFxrID2-Lq0pJiDJPmN6diok,887,qu1khyGSlXRnquhyIFIPNg677yuBYVfTSG0R585tlguy9cTA,234,7nywNCOnXyibsU3cz-I62uZN3w2b5JtbfA4vE9_bWPShRBg,55,4jvxeV9D7EX20zvgJ0vAMMryB4YOMuxzZP3pRSaHsR5YP7...,236,oIVAMaNkewIQQPvb7MhWtHFWUxPXQ2sqnsaHpFfD1mtnHamM,...,BLUE,1,4,NONE,0,0,NONE,0,0,RED
0,EUW1_6001565779,-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww,113,ThLj1asjdN0UDYhws7Nr6nt8aPbMbh8d___jxW_VY2IhEht_,106,RuS52KcQmxnoGwOSF-77bCn7XumUV2cDitH361Ehlhr7BB...,38,AlWFNGSymuqUZGV6ZJvoK4BxcrIyIxfkhL4MbmuF6IDMBH...,895,TFrqUhhsR6ihU8X1n5cqviWKTLRFINR_b-ePdm3TXIJu7lo,...,RED,3,10,RED,0,1,RED,0,1,RED
0,EUW1_6001582598,-0xKeKNvTNUPuswOsqGYJVhcBlx-ydIhH_wJ30Y-1_44Iww,58,Zrk6U6YEPOjholD92lHxXHmmULSLJpehPdZBNkCgigbWZZfX,62,IdG1r86PVfePXywMZov6jrphqQFgASkUiy56xitBqrQFsFHp,84,edOZSEUq2zlUwoPrp78rryMIcmxoc2nX1NeXvzsnSqL39TaE,81,m_3SLhHTOp7hG1Av6ZyjY8YM9DG7mQusgcrtzwyDGRuadvg,...,BLUE,10,5,BLUE,1,0,BLUE,2,0,BLUE
0,EUW1_6001039285,Z9T9hdDGsFsESNS8H5TQbHkZvbhYKtULGGlNdopJ6JobVyo,113,WCcraRJ8siFOsMPHQULiTNs61kWxmQBCJlX1QK66rxPy9Fgb,254,gVHrLOuqtwFQ4p3fjL_Gq4K5Y6vfkqmk43kpUKz8AopRlB4,112,fZhXXeYxKZMewEOKerIesPnZ6Y6bgCwRb7j9e69Oeb4N9b8,895,1_P5Q8O0Nar7kF8IbbS2YyK0ShzFiL0k6fRb_lrchUhtge...,...,RED,4,7,RED,0,1,RED,0,1,RED


In [24]:
matches.shape

(20, 50)